# Criação de dataset de obrigações


## Configuração, extração e limpeza dos dados

In [32]:
import os
import pymssql

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from dotenv import load_dotenv

load_dotenv()

def get_connection(db='processo'):
    return pymssql.connect(
        server=os.getenv("SQL_SERVER_HOST"),
        user=os.getenv("SQL_SERVER_USER"),
        password=os.getenv("SQL_SERVER_PASS"),
        port=os.getenv("SQL_SERVER_PORT"),
        database=db
    )



from langchain_openai import AzureOpenAI, AzureChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate, FewShotChatMessagePromptTemplate

llm = AzureChatOpenAI(model_name="gpt-4o")


In [33]:
llm

AzureChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x7432dd4d6420>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x7432de138fe0>, root_client=<openai.lib.azure.AzureOpenAI object at 0x7432dd9253a0>, root_async_client=<openai.lib.azure.AsyncAzureOpenAI object at 0x7432dd61cad0>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'), azure_endpoint='https://recursoopenai-dip.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-03-01-preview', openai_api_version='2025-03-01-preview', openai_api_type='azure')

In [43]:
sql_decisoes = """ 
SELECT 
IdComposicaoPauta as iddecisao,
CONCAT(acp.numero_processo, '/', acp.ano_processo) as processo, 
t.codigo as tipo_processo,
t.descricao as descricao_tipo_processo,
acp.* 
FROM processo.dbo.Ata_Composicao_Pautas acp 
	INNER JOIN processo.dbo.Processos pro 
	ON acp.numero_processo = pro.numero_processo AND acp.ano_processo = pro.ano_processo
	INNER JOIN processo.dbo.Tipo t
	ON t.codigo = pro.codigo_tipo_processo

WHERE ( year(acp.DataPublicacaoDOE) = 2024 AND month(acp.DataPublicacaoDOE) IN (10, 11, 12) )
AND t.codigo NOT IN ('APO', 'NCE', 'PEN', 'INM', 'ASS', 'CTT')
AND acp.resultado is not null
"""
df_decisoes_20244 = pd.read_sql(sql_decisoes, get_connection())

sql_decisoes = """ 
SELECT 
IdComposicaoPauta as iddecisao,
CONCAT(acp.numero_processo, '/', acp.ano_processo) as processo, 
t.codigo as tipo_processo,
t.descricao as descricao_tipo_processo,
acp.* 
FROM processo.dbo.Ata_Composicao_Pautas acp 
	INNER JOIN processo.dbo.Processos pro 
	ON acp.numero_processo = pro.numero_processo AND acp.ano_processo = pro.ano_processo
	INNER JOIN processo.dbo.Tipo t
	ON t.codigo = pro.codigo_tipo_processo

WHERE ( year(acp.DataPublicacaoDOE) = 2025 AND month(acp.DataPublicacaoDOE) IN (1, 2, 3) )
AND t.codigo NOT IN ('APO', 'NCE', 'PEN', 'INM', 'ASS', 'CTT')
AND acp.resultado is not null
"""
df_decisoes_20251 = pd.read_sql(sql_decisoes, get_connection())


/tmp/ipykernel_26085/309705961.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_decisoes_20244 = pd.read_sql(sql_decisoes, get_connection())
/tmp/ipykernel_26085/309705961.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_decisoes_20251 = pd.read_sql(sql_decisoes, get_connection())


In [44]:
len(df_decisoes_20244), len(df_decisoes_20251)

(493, 375)

In [45]:
retirar_tipos = ['APO', 'NCE', 'PEN', 'INM', 'ASS', 'FCO']
df_decisoes_20251 = df_decisoes_20251[~df_decisoes_20251.tipo_processo.isin(retirar_tipos)]

In [46]:
df_decisoes_20251

/home/eduardo/Dev/ccd/.venv/lib/python3.12/site-packages/IPython/core/formatters.py:406: FutureWarning: Index.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.
  return method()


,iddecisao,processo,tipo_processo,descricao_tipo_processo,IdComposicaoPauta,codigo_camara,numero_sessao,ano_sessao,numero_processo,ano_processo,...,UsuarioLiberadoPublicacaoMonocratica,IdMotivoRetirada,IdApreciacao,idTipoRecurso,IdTipoVotacao,POssuiObrigacao,IdTipoIrregularidade,PlenarioVirtualVistasMPC,PlenarioVirtualSustentacaoOralMpc,ContemVotoVista
0,118661,004761/2024,ADM,PROCESSOS ADMINISTRATIVOS ...,118661,3,00001,2025,004761,2024,...,None,None,2,NaN,1.0,False,NaN,None,None,None
1,118432,003595/2022,ADM,PROCESSOS ADMINISTRATIVOS ...,118432,3,0004E,2024,003595,2022,...,None,None,2,NaN,1.0,False,NaN,None,None,None
2,119088,003702/2018,ADM,PROCESSOS ADMINISTRATIVOS ...,119088,3,00003,2025,003702,2018,...,None,None,2,NaN,1.0,False,NaN,None,None,None
3,118779,004875/2020,ADM,PROCESSOS ADMINISTRATIVOS ...,118779,3,00002,2025,004875,2020,...,None,None,2,NaN,1.0,False,NaN,None,None,None
4,118780,004876/2020,ADM,PROCESSOS ADMINISTRATIVOS ...,118780,3,00002,2025,004876,2020,...,None,None,2,NaN,1.0,False,NaN,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370,119092,015674/2002,REP,REPRESENTAÇÃO ...,119092,3,0004v,2025,015674,2002,...,None,None,2,NaN,1.0,False,NaN,None,None,None
371,119240,005857/2012,REP,REPRESENTAÇÃO ...,119240,3,0004V,2025,005857,2012,...,None,None,2,NaN,1.0,False,NaN,None,None,None
372,118922,003792/2010,REP,REPRESENTAÇÃO ...,118922,1,0003V,2025,003792,2010,...,None,None,2,NaN,1.0,False,NaN,None,None,None
373,118853,002333/2003,TAD,TERMO ADITIVO ...,118853,3,0003V,2025,002333,2003,...,None,None,2,NaN,1.0,False,NaN,None,None,None


## Análise dos dados

In [38]:
df_decisoes_20251

/home/eduardo/Dev/ccd/.venv/lib/python3.12/site-packages/IPython/core/formatters.py:406: FutureWarning: Index.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.
  return method()


,iddecisao,processo,tipo_processo,descricao_tipo_processo,IdComposicaoPauta,codigo_camara,numero_sessao,ano_sessao,numero_processo,ano_processo,...,UsuarioLiberadoPublicacaoMonocratica,IdMotivoRetirada,IdApreciacao,idTipoRecurso,IdTipoVotacao,POssuiObrigacao,IdTipoIrregularidade,PlenarioVirtualVistasMPC,PlenarioVirtualSustentacaoOralMpc,ContemVotoVista
0,118661,004761/2024,ADM,PROCESSOS ADMINISTRATIVOS ...,118661,3,00001,2025,004761,2024,...,None,None,2,NaN,1.0,False,NaN,None,None,None
1,118432,003595/2022,ADM,PROCESSOS ADMINISTRATIVOS ...,118432,3,0004E,2024,003595,2022,...,None,None,2,NaN,1.0,False,NaN,None,None,None
2,118779,004875/2020,ADM,PROCESSOS ADMINISTRATIVOS ...,118779,3,00002,2025,004875,2020,...,None,None,2,NaN,1.0,False,NaN,None,None,None
3,118780,004876/2020,ADM,PROCESSOS ADMINISTRATIVOS ...,118780,3,00002,2025,004876,2020,...,None,None,2,NaN,1.0,False,NaN,None,None,None
4,119088,003702/2018,ADM,PROCESSOS ADMINISTRATIVOS ...,119088,3,00003,2025,003702,2018,...,None,None,2,NaN,1.0,False,NaN,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371,118761,005051/2020,REP,REPRESENTAÇÃO ...,118761,3,0002V,2025,005051,2020,...,None,None,2,NaN,1.0,False,NaN,None,None,None
372,118702,003302/2020,REP,REPRESENTAÇÃO ...,118702,1,00002,2025,003302,2020,...,None,None,2,NaN,3.0,True,3.0,None,None,True
373,118557,002529/2024,REP,REPRESENTAÇÃO ...,118557,1,0001V,2025,002529,2024,...,None,None,2,NaN,1.0,True,NaN,None,None,None
374,118853,002333/2003,TAD,TERMO ADITIVO ...,118853,3,0003V,2025,002333,2003,...,None,None,2,NaN,1.0,False,NaN,None,None,None


In [39]:
df_decisoes_20251

/home/eduardo/Dev/ccd/.venv/lib/python3.12/site-packages/IPython/core/formatters.py:406: FutureWarning: Index.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.
  return method()


,iddecisao,processo,tipo_processo,descricao_tipo_processo,IdComposicaoPauta,codigo_camara,numero_sessao,ano_sessao,numero_processo,ano_processo,...,UsuarioLiberadoPublicacaoMonocratica,IdMotivoRetirada,IdApreciacao,idTipoRecurso,IdTipoVotacao,POssuiObrigacao,IdTipoIrregularidade,PlenarioVirtualVistasMPC,PlenarioVirtualSustentacaoOralMpc,ContemVotoVista
0,118661,004761/2024,ADM,PROCESSOS ADMINISTRATIVOS ...,118661,3,00001,2025,004761,2024,...,None,None,2,NaN,1.0,False,NaN,None,None,None
1,118432,003595/2022,ADM,PROCESSOS ADMINISTRATIVOS ...,118432,3,0004E,2024,003595,2022,...,None,None,2,NaN,1.0,False,NaN,None,None,None
2,118779,004875/2020,ADM,PROCESSOS ADMINISTRATIVOS ...,118779,3,00002,2025,004875,2020,...,None,None,2,NaN,1.0,False,NaN,None,None,None
3,118780,004876/2020,ADM,PROCESSOS ADMINISTRATIVOS ...,118780,3,00002,2025,004876,2020,...,None,None,2,NaN,1.0,False,NaN,None,None,None
4,119088,003702/2018,ADM,PROCESSOS ADMINISTRATIVOS ...,119088,3,00003,2025,003702,2018,...,None,None,2,NaN,1.0,False,NaN,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371,118761,005051/2020,REP,REPRESENTAÇÃO ...,118761,3,0002V,2025,005051,2020,...,None,None,2,NaN,1.0,False,NaN,None,None,None
372,118702,003302/2020,REP,REPRESENTAÇÃO ...,118702,1,00002,2025,003302,2020,...,None,None,2,NaN,3.0,True,3.0,None,None,True
373,118557,002529/2024,REP,REPRESENTAÇÃO ...,118557,1,0001V,2025,002529,2024,...,None,None,2,NaN,1.0,True,NaN,None,None,None
374,118853,002333/2003,TAD,TERMO ADITIVO ...,118853,3,0003V,2025,002333,2003,...,None,None,2,NaN,1.0,False,NaN,None,None,None


In [40]:
df_decisoes = df_decisoes_20251[~df_decisoes_20251['resultado'].isna()]
df_decisoes_20251['resultado'].fillna('', inplace=True)

/tmp/ipykernel_26085/2363936717.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_decisoes_20251['resultado'].fillna('', inplace=True)


## Funções LLM

In [47]:
def classify_decision(text):
    examples = [
        {
            "input": """
Apresento aos Excelentíssimos Srs. Conselheiros Integrantes desta Segunda Câmara de Contas, Voto nos termos a seguir delineados:
1)	Emitir PARECER PRÉVIO PELA DESAPROVAÇÃO DAS CONTAS, relativas ao exercício de 2015, da PREFEITURA MUNICIPAL DE JUCURUTU/RN, sob a responsabilidade do Excelentíssimo Senhor Prefeito Sr. George Retlen Costa Queiroz, submetendo-o à Augusta Câmara de Vereadores do referido Município, cuja deliberação deve ser oportunamente informada a este Tribunal;
2)	Determinar a imediata remessa de cópias do presente Parecer Prévio ao Ministério Público Estadual, para que possa, no âmbito de suas atribuições constitucionais e legais, apurar os fatos pelos quais se emite o presente Parecer Prévio;
3)	Determinar a instauração de processo de Apuração de Responsabilidade perante esta Corte, oriundo desta mesma relação processual e sob a relatoria deste Conselheiro, em face do Chefe do Poder Executivo, época, Excelentíssimo Senhor Prefeito George Retlen Costa Queiroz, em razão das seguintes irregularidades: Não aplicação do mínimo constitucional em educação; descumprimento da meta de resultado primário estipulado na LDO/2015, em razão do não contingenciamento de despesas ao longo da execução do orçamento durante o exercício; ausência de decretos e/ou do QDD atualizado para abertura de créditos suplementares, não remessa, ao TCE/RN, de alguns documentos e informações exigidos pelos arts. 10 e 11 da Resolução n.º 04/2013-TCE  (caráter reincidente da irregularidade, vez que também foi evidenciada na PCA de 2014, sob a responsabilidade do mesmo gestor, no âmbito do Processo n.º 6537/2015-TC); divergência dos valores informados na contabilidade com os dados prestados ao SIOPS relativos aos gastos com saúde, impossibilitando atestar o percentual de gastos em ações e serviços de saúde, para fins de cumprimento do disposto no art. 198, § 2º, III, e art. 77, III, ADCT, ambos da CF/88, com a posterior remessa do processo de apuração de responsabilidade constituído ao Ministério Público de Contas para pronunciamento, tudo nos moldes do artigo 247-B do Regimento Interno deste Tribunal (Resolução nº 009/2012-TCE), incluído pela Resolução nº 012/2016-TCE;
4)	Determinar a expedição de recomendação ao atual Chefe do Poder Executivo em referência para que adote medidas necessárias à melhoria da qualidade das informações contábeis;
5)	Esclarecer que as conclusões deste Parecer não excluem o julgamento, por este Tribunal, das Contas individualizadas de responsabilidade dos ordenadores de despesa e demais responsáveis por dinheiros, bens e valores públicos;
            """,
            "output": "DETERMINACAO"}, {"input": """
            - Adoção das providências cabíveis no tocante aos indícios de impropriedades /irregularidades
    elencadas na tabela 19 do Relatório (transcritas acima ), com fundamento no art. 2º, inciso III, da
    Resolução nº 012/2023-TCE, notadamente para que a Secretaria de Controle Externo analise a
    capacidade operacional desta Corte com vistas ao acompanhamento e /ou à abertura de processo
    autônomo, respeitadas a conveniência e oportunidade, referente aos itens 4.2 (Divergência dos
    dados do planejamento do Projeto, Inconsistência dos saldos da conta bens móveis entre os
    registros do SIGEF - Sistema Integrado de Planejamento e Gestão Fiscal - e SMI - Sistema de
    Monitoramento e Informações do Projeto - e deficiências na elaboração das notas explicativas ),
    5.5.1.1 (Inventário físico ), 5.6.1.1 ( Falhas/deficiências construtivas do Hospital Regional da
    Mulher Parteira Maria Correia) e 5.6.1.3 (Pendências de legalização das obras da Biblioteca
    Câmara Cascudo e da Sede do Serviço Nacional de Emprego – SINE).""",
            "output": "DETERMINACAO"}, {
            "input": """( a.i) promovam, no prazo de 120 (cento e vinte dias) úteis, contados a partir da intimação da
    presente Decisão, a apuração dos fatos e se verifique a constitucionalidade e legalidade dos
    vínculos funcionais de cada servidor que figura nos Anexos nºs. 01 e 02 contidos nos Eventos
    nºs. 04 e 05, além de outros que porventura sejam informados pela DDP em cumprimento ao
    item b, por meio da instauração de processos administrativos disciplinares individuais ,
    regulados pela Lei que trata do Estatuto Jurídico dos Servidores do respectivo Município ,
    com observância dos princípios do contraditório, ampla defesa e devido processo legal;
        ( a.ii) comprovem neste feito, em 05 dias úteis após ultimado o prazo de definição dos PAD
    ´s, as conclusões de todos os processos administrativos instaurados, no tocante à eliminação
    de tríplice vínculo funcional identificado e de enquadramento das eventuais acumulações
    dúplices nas hipóteses permitidas pela Constituição Federal, com a respectiva
    compatibilidade de horários, sob pena de, não cumprindo tais obrigações nos prazos antes
    referidos, incidir em multa diária e pessoal ao gestor, no valor de R$ 500,00, com espeque no
    art. 110 da LCE nº 464/2012 c/c o art. 326 do RITCE, cabendo à Diretoria de Despesa com
    Pessoal monitorar o cumprimento da presente Decisão;""",
            "output": "DETERMINACAO"}, {
            "input": """Vistos, relatados e discutidos estes autos, em consonância ao posicionamento do
    Corpo técnico e do Ministério Público de Contas, ACORDAM os Conselheiros, nos termos
    do voto proposto pelo Conselheiro Relator, julgar a inadmissibilidade da presente denúncia e
    o seu conseqüente arquivamento, com fulcro nos art. 12 do Provimento 002/2020 –
    CORREG/TCE, aprovado pela Resolução 016/2020 – TCE e artigo 80, § 1º, da LOTCE.
    E ainda, Pela expedição de RECOMENDAÇÃO, nos termos do art. 301, III da Resolução
    009/2012 (RITCE/RN) c/c art. 13, II da Resolução 16/2020 –TCE/RN, ao Executivo
    Municipal de Nísia Floresta /RN, com cópia para o respectivo órgão de controle interno, ou
    setor responsável pelas publicações oficiais, lastreada na Constituição Federal de 88/ Art. 37,
    a fim de que promova e deixe claro os seguintes comportamentos em suas postagens""",
            "output": "DETERMINACAO"}, {
            "input": """
    Ante o exposto, discordando do Corpo Técnico da DAM e do Ministério Público de Contas – que se manifestaram no sentido de julgar o mérito do feito pela aprovação da matéria – VOTO no sentido de:
    1) preliminarmente, considero inexistentes os documentos constantes no processo apensado n. 300799/2024, na forma do art. 166, § 2º, do RITCE;
    2) rejeitar a preliminar de incompetência desta Corte de Contas para a análise das contratações objeto dos autos;
    3) nos termos dos arts. 120 e 121 da LCE nº 464/2012, conceder, de ofício, tutela provisória (medida cautelar) para determinar aos atuais gestores dos Municípios de Lagoa Nova/RN, Cruzeta/RN e Jucurutu/RN, que:
    a) tão logo sejam intimados, suspendam de forma imediata os pagamentos de honorários advocatícios em favor de Fernanda de Paula Sociedade Individual de Advocacia, decorrentes dos Contratos objeto dos autos, ainda que encerrada a sua vigência, ou de termos aditivos destes, acaso existentes, até o julgamento de mérito do presente feito, sob pena de multa pessoal ao gestor recalcitrante no valor de R$ 50.000,00 (cinquenta mil reais) por cada novo pagamento realizado;
    b) assim que forem intimados, promovam, também de forma imediata e por meio de ato formal, a habilitação das Procuradorias de cada um dos Municípios nos autos dos processos administrativos e/ou judiciais nos quais Fernanda de Paula Sociedade Individual de Advocacia representa os Entes municipais;
    c) no prazo de 10 (dez) dias úteis, a contar de sua intimação, comprovem nestes autos o exato cumprimento das medidas cautelares ora determinadas, especialmente no que tange à suspensão imediata dos pagamentos e à habilitação, também imediata, das Procuradorias nos processos que Fernanda de Paula Sociedade Individual de Advocacia representa os respectivos Entes municipais. No mesmo prazo, devem juntar ao presente feito eventuais termos aditivos realizados com Fernanda de Paula Sociedade Individual de Advocacia, ou comprovar que não houve, sob pena de multa pessoal ao gestor recalcitrante no valor de R$ 5.000,00 (cinco mil reais) por cada dia de eventual descumprimento destas ordens, na forma do art. 110, da LCE nº 464/2012;
    d) com base no art. 82, inciso V, da Lei Complementar Estadual n. 464/12, e no art. 288, do Regimento Interno do TCE/RN, pela determinação do MONITORAMENTO do cumprimento da presente Decisão, por intermédio da DAM;
    4) a citação dos Srs. Iogo Nielson de Queiroz e Silva, Luciano Silva Santos e Joaquim José de Medeiros, Prefeitos Municipais de Jucurutu/RN, Lagoa Nova/RN e Cruzeta/RN, respectivamente, além de Fernanda de Paula Sociedade Individual de Advocacia, com arrimo nos artigos 37, c/c 45, §1º, inciso II, ambos da Lei Complementar Estadual nº 464/2012, a fim de que, querendo, apresentem suas razões de defesa;
    5) a remessa imediata de cópia da decisão ora proferida ao Ministério Público Comum Estadual, haja vista a probabilidade de dano ao erário em razão dos pagamentos já realizados e que seguem acontecendo, possibilitando-lhe o conhecimento para eventuais providências no âmbito de sua competência.
    Intimem-se da forma mais célere possível, nos termos do art. 46, § 4º da LCE 464/2012, além dos atuais gestores dos Municípios de Lagoa Nova/RN, Cruzeta/RN e Jucurutu/RN, também a Representada Fernanda de Paula Sociedade Individual de Advocacia.
    Sala das Sessões, data da assinatura eletrônica.

    (assinado digitalmente)
    Carlos Thompson Costa Fernandes
    Conselheiro Relator
    """,
            "output": "DETERMINACAO"}, {
            "input": """
            Pelo exposto, concordando parcialmente com o Representante e em dissonância com o Ministério Público de Contas – divirjo do Representante em relação ao prazo proposto e deste último por entender pelo deferimento do pedido cautelar e não pela citação da gestora –, VOTO:

    a) pelo acolhimento em parte do pedido cautelar apontado no item “b” da Exordial, ao passo que determino a intimação do Município de Tenente Ananias/RN e de sua gestora para que:

    (a.i) promovam, no prazo de 120 (cento e vinte dias) úteis, contados a partir da intimação da presente Decisão, a apuração dos fatos e se verifique a constitucionalidade e legalidade dos vínculos funcionais de cada servidor que figura na Informação Técnica da DDP junto ao evento nº 33, por meio da instauração de processos administrativos disciplinares individuais, regulados pela Lei que trata do Estatuto Jurídico dos Servidores do respectivo Município, com observância dos princípios do contraditório, ampla defesa e devido processo legal;

    (a.ii) comprovem neste feito, em 05 dias úteis após ultimado o prazo de definição dos PAD´s, as conclusões de todos os processos administrativos instaurados, no tocante à eliminação de tríplice vínculo funcional identificado e de enquadramento das eventuais acumulações dúplices nas hipóteses permitidas pela Constituição Federal, com a respectiva compatibilidade de horários, sob pena de, não cumprindo tais obrigações nos prazos antes referidos, incidir em multa diária e pessoal à gestora, no valor de R$ 500,00, com espeque no art. 110 da LCE nº 464/2012 c/c o art. 326 do RITCE, cabendo à Diretoria de Despesa com Pessoal monitorar o cumprimento da presente Decisão;

    b) com base no art. 82, inciso V, da Lei Complementar Estadual nº 464/12, e no art. 288 do Regimento Interno do TCE/RN, pela determinação do MONITORAMENTO do cumprimento da presente Decisão, por intermédio da DDP.


    (assinado digitalmente)
    Carlos Thompson Costa Fernandes
    Conselheiro Relator
            """,
            "output": "DETERMINACAO"},{
                "input": """Vistos, relatados e discutidos estes autos, acolhendo os fundamentos do parecer
    ministerial, com substrato no art. 209 V da norma regimental, ACORDAM os Conselheiros ,
    nos termos do voto proposto pela Conselheira Relatora, julgar pelo ARQUIVAMENTO dos
    autos.""",
                "output": "OUTROS",
            },
            {
                "input": """Vistos, relatados e discutidos estes autos, ACORDAM os Conselheiros,  com o
    impedimento do Conselheiro Presidente Renato Costa Dias, nos termos do voto profposto
    pela Conselheira Relatora, haja vista os fundamentos fático -jurídicos explanados no excerto
    antecedente, comprovado documentalmente o adimplemento substancial do plano de
    redimensionamento/adequação do sistema de ensino natalense, julgar pela EXTINÇÃO do
    FEITO nos termos do art. 71 da Lei Complementar (estadual) c/c art. 22 §1° da LINDB e art .
    209 V da regra regimental.""",
                "output": "OUTROS",
            },
            {
                "input": """Vistos, relatados e discutidos estes autos, em consonância com o posicionamento da
    Diretoria de Administração Municipal – DAM e do Ministério Público de Contas ,
    ACORDAM os Conselheiros, nos termos do voto proferido pelo Conselheiro Relator, julgar
    pelo reconhecimento da incidência da Prescrição Intercorrente sobre a pretensão punitiva e
    ressarcitória desta Corte de Contas, nos termos do artigo 111, parágrafo único, da Lei
    Complementar Estadual nº 464/2012, com o consequente arquivamento dos presentes autos.
    E ainda, pelo envio de cópia das principais peças dos autos ao Ministério Público Estadual ,
    para conhecimento e atuação no âmbito de sua competência.""",
                "output": "OUTROS",
            },
            {
                "input": """Vistos, relatados e discutidos estes autos, em dissonância com o Ministério Público
    de Contas, ACORDAM os Conselheiros, nos termos do voto proferido pelo Conselheiro
    Relator, julgar pela extinção do feito, sem julgamento de mérito, com o consequente
    ARQUIV AMENTO dos autos, em virtude da incompatibilidade entre a emissão de parecer
    prévio pela aprovação com ressalvas das contas de governo e a instauração de apuração de
    responsabilidade para aplicação de sanções.""",
                "output": "OUTROS",
            }
        ]

    example_prompt = ChatPromptTemplate.from_messages([
            ("human", '''
            {input} '''),
            ("ai", "{output}"),
        ])

    few_shot_prompt = FewShotChatMessagePromptTemplate(
        example_prompt=example_prompt,
        examples=examples,)

    final_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", """Você é um classificador de decisões de um tribunal de contas.
            Sua tarefa é definir se uma decisão trata de determinação de obrigação ou recomendação de fazer ou não.            

            Responda com DETERMINACAO se o texto contiver alguma recomendação ou obrigação de fazer, independente de haver multa ou não
            Responda com OUTROS se o texto tratar de arquivamento, extinção do feito ou prescrição da matéria, ou outro assunto que não seja DETERMINACAO

            Responda APENAS com DETERMINACAO ou OUTROS. Os únicos valores possíveis são DETERMINACAO ou OUTROS."""),
            few_shot_prompt,
            ("human", "{input}"),])
    
    chain = final_prompt | llm
    decision_type = chain.invoke({"input": text}).content
    return decision_type

In [48]:
for i,decisao in df_decisoes_20251.iterrows():
    classf = classify_decision(decisao['resultado'])
    # insert classification into ClassificacaoDecisao SQL SERVER table
    with get_connection("BdDIP") as conn:
        with conn.cursor() as cursor:
            cursor.execute(f"""
            SELECT COUNT(*) FROM ClassificacaoDecisao WHERE idDecisao = {decisao['iddecisao']}
            """)
            count = cursor.fetchone()[0]
            if count == 0:
                cursor.execute(f"""
                INSERT INTO ClassificacaoDecisao (idDecisao, Classificacao) VALUES ({decisao['iddecisao']}, '{classf}')
                """)
                conn.commit()
            else:
                cursor.execute(f"""
                UPDATE ClassificacaoDecisao SET Classificacao = '{classf}' WHERE idDecisao = {decisao['iddecisao']}
                """)
                conn.commit()                
    print(f"{i} - {decisao['iddecisao']} - {classf}")

0 - 118661 - OUTROS
1 - 118432 - OUTROS
2 - 119088 - OUTROS
3 - 118779 - OUTROS
4 - 118780 - OUTROS
5 - 119462 - OUTROS
6 - 119219 - OUTROS
7 - 119266 - OUTROS
8 - 119063 - OUTROS
9 - 119223 - OUTROS
10 - 118774 - DETERMINACAO
11 - 118879 - DETERMINACAO
12 - 117748 - OUTROS
13 - 118196 - OUTROS
14 - 118221 - DETERMINACAO
15 - 118233 - DETERMINACAO
16 - 118551 - DETERMINACAO
17 - 118494 - DETERMINACAO
18 - 118508 - DETERMINACAO
19 - 118418 - OUTROS
20 - 118574 - OUTROS
21 - 118507 - DETERMINACAO
22 - 119192 - OUTROS
23 - 118792 - OUTROS
24 - 119130 - OUTROS
25 - 118652 - DETERMINACAO
26 - 118653 - DETERMINACAO
27 - 118654 - DETERMINACAO
28 - 118655 - DETERMINACAO
29 - 118638 - OUTROS
30 - 118549 - OUTROS
31 - 118745 - OUTROS
32 - 118746 - OUTROS
33 - 118710 - OUTROS
34 - 118718 - OUTROS
35 - 118719 - DETERMINACAO
36 - 118720 - OUTROS
37 - 118722 - OUTROS
38 - 118769 - DETERMINACAO
39 - 118770 - DETERMINACAO
40 - 118771 - DETERMINACAO
41 - 118772 - DETERMINACAO
42 - 118773 - DETERMINACAO

## Recuperação dos classificados

In [49]:
sql_classificado_20251 = f""" 
SELECT IdDecisao, Classificacao FROM ClassificacaoDecisao
WHERE IdDecisao IN ({', '.join(df_decisoes_20251['iddecisao'].unique().astype(str).tolist())})
"""
df_classificado_20251 = pd.read_sql(sql_classificado_20251, get_connection("BdDIP"))

/tmp/ipykernel_26085/1894815480.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_classificado_20251 = pd.read_sql(sql_classificado_20251, get_connection("BdDIP"))


In [50]:
determinacoes_20251 = df_classificado_20251[df_classificado_20251['Classificacao'].isin(['DETERMINACAO'])]

In [55]:
df_classificado_20251.Classificacao.value_counts()

Classificacao
DETERMINACAO    198
OUTROS          177
Name: count, dtype: int64

## Extração de dados

In [56]:
from typing import Optional, List
from pydantic import BaseModel, Field

class Obrigacao(BaseModel):
    """Determinação de obrigação ou recomendação, com valor se houver, prazo se houver, e responsável"""
    responsavel: str = Field(description="Nome do órgão ou gestor responsável pela obrigação")
    prazo: str = Field(description="Prazo para cumprimento da obrigação")
    valor: Optional[float] = Field(description="Valor da multa, se houver")
    descricao: str = Field(description="Texto descritivo da obrigação")

class Recomendacao(BaseModel):
    """Recomendação do TCE/RN com prazo se houver e responsável"""
    responsavel: str = Field(description="Nome do órgão ou gestor responsável pela recomendação")
    prazo: str = Field(description="Prazo para cumprimento da recomendação")
    descricao: str = Field(description="Texto descritivo da recomendação")

class Decisao(BaseModel):
    """Decisão processual do TCE/RN"""
    determinacao: List[Optional[Obrigacao|Recomendacao]] = Field(description="Determinação de obrigações")


In [57]:
def extract_decision_information(decision):
    prompt = PromptTemplate.from_template("""
  Você é um agente que identifica listas de determinações em textos de decisões. Seu objetivo
  é extrair um conjunto de textos de uma lista que contém obrigações ou recomendações. Se não houver
  determinações, responda apenas com "N/D". Ignore aplicações de multas.

  Decisão : {input}

  Sua resposta:
  """)

    structured_llm = llm.with_structured_output(schema=Decisao, method="function_calling")
    chain = prompt | structured_llm
    response = chain.invoke({"input": decision})
    return response

In [58]:
determinacoes = df_decisoes_20251[df_decisoes_20251['iddecisao'].isin(determinacoes_20251['IdDecisao'].unique())]
for i, determinacao in determinacoes.iterrows():
    # extract decision information
    try:
        decision_info = extract_decision_information(determinacao['resultado'])
    except:
        print(f"Erro ao processar a decisão {determinacao['iddecisao']}")
        continue

    for d in decision_info.determinacao:
        if not d:
            continue

        erecomendacao = isinstance(d, Recomendacao)
        with get_connection("BdDIP") as conn:
            with conn.cursor() as cursor:
                # Insert into the table ObrigacaoProcessada
                # INSERT INTO BdDIP.dbo.ObrigacaoPreprocessada(ERecomendacao, Valor, IdObrigacao, IdDecisao, Prazo, Descricao, OrgaoResponsavel, Validado)
                valor = d.valor if isinstance(d, Obrigacao) else ''

                cursor.execute(f"""
                SELECT COUNT(*) FROM ObrigacaoPreprocessada WHERE IdDecisao = {determinacao['iddecisao']}
                """)
                count = cursor.fetchone()[0]
                if count == 0:
                    sql_comnd = f"""
                    INSERT INTO ObrigacaoPreprocessada (ERecomendacao, Valor, IdDecisao, Prazo, Descricao, OrgaoResponsavel, Validado)
                    VALUES ({int(erecomendacao)}, '{valor}', {determinacao['iddecisao']}, '{d.prazo}', '{d.descricao}', '{d.responsavel}', 0)
                    """
                    #print(sql_comnd)
                    cursor.execute(sql_comnd)
                    conn.commit()
                else:
                    cursor.execute(f"""
                    UPDATE ObrigacaoPreprocessada SET ERecomendacao = {int(erecomendacao)}, Valor = '{valor}', Prazo = '{d.prazo}', Descricao = '{d.descricao}', OrgaoResponsavel = '{d.responsavel}' WHERE IdDecisao = {determinacao['iddecisao']} AND Descricao = '{d.descricao}'
                    """)
                    conn.commit()
                print(f"{i} - {determinacao['iddecisao']} - {d.descricao} - {d.responsavel} - {d.prazo} - {valor}")

                        

10 - 118774 - Adotar procedimentos com vistas a impedir a consumação de prescrição nas Unidades Técnicas a ela vinculadas - Secretaria de Controle Externo – SECEX -  - 
11 - 118879 - Elaborar um plano de ação para cumprimento das recomendações abaixo descritas - Secretaria de Estado das Mulheres, da Juventude, da Igualdade Racial e dos Direitos Humanos (SEMJIDH) - 60 dias - 
11 - 118879 - Realizar um levantamento dos normativos que ainda não foram efetivamente implementados, com objetivo de articular a proposição de ações relativas a essas lacunas junto aos órgãos executores - SEMJIDH -  - 
11 - 118879 - Monitorar as ações regulamentadas por normativos relacionadas ao combate a violência contra as mulheres, tanto no nível de instância de coordenação do governo do Estado, quanto no nível de presidência/participante dos conselhos e comitês - SEMJIDH -  - 
11 - 118879 - Realizar estudos, pesquisas e coleta de dados estatísticos abrangendo todos os territórios do Rio Grande do Norte, com o